# Data Visualization Module
The purpose of this notebook is to act as a module that can be imported into the Master Notebook to visualize various data.

In [16]:
def getFinePMTimeSeries(x, y, ylabel, figure_size=(16,8)):
    '''
    Inputs:
        - x: numpy DateTime array holding the independent variable data
        - y: numpy float array holding the PM2.5 concentration or AQI
        - ylabel: string representing the y-axis label 
        - figure_size: a tuple holding the figure size; default is (16,8)
    Returns a time series plot of PM2.5 Concentration or AQI with AQI breakpoints included as background
    '''
    # Plotting
    fig, ax = plt.subplots(figsize = figure_size)
    ax.plot(x,y,color='black',linewidth=3,label='PM2.5')

    ## Formatting x-axis
    ax.set_xlim([x[0],x[-1]])
    ### Formatting datetime labels
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d %H:%M'))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_minor_locator(mdates.HourLocator(byhour=range(2,24,2)))
    ax.set_xlabel('Time')
    ax.xaxis.set_tick_params(rotation=-30)
    
    ## Formatting y-axis
    ax.set_ylim([0,500])
    ax.set_ylabel(ylabel)
    
    ## Formatting remaining aspects of figure
    ax.set_title('Day: ' + str(x[2].date()))
    ### AQI breakpoints
    ax.axhspan(0,12,color='green',alpha=0.3,label='Good')
    ax.axhspan(12,35.4,color='yellow',alpha=0.3,label='Moderate')
    ax.axhspan(35.5,55.4,color='orange',alpha=0.3,label='Unhealthy for Children/Elderly')
    ax.axhspan(55.5,150.4,color='red',alpha=0.3,label='Unhealthy')
    ax.axhspan(150.5,250.4,color='purple',alpha=0.3,label='Very Unhealthy')
    ax.axhspan(250.5,500,color='maroon',alpha=0.3,label='Hazardous')
    ax.legend()